## КТ4

### Базовый класс

In [46]:
%%file Base_Page.py
from selenium import webdriver


class BasePage:
    BASE_URL = "http://192.168.0.225:8081"

    def __init__(self, driver):
        self.driver = driver
        self.driver.implicitly_wait(3)
        self.driver.get(BasePage.BASE_URL)

Overwriting Base_Page.py


### Тесты для главной страницы

In [47]:
%%file test_Main_Page.py
from Base_Page import BasePage
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException
from selenium import webdriver
import pytest
import allure


class MainPage(BasePage):
    FEATURED_PRODUCT = (By.CSS_SELECTOR, "#content > div.row .product-thumb")
    PRODUCT_NAME = (By.CSS_SELECTOR, ".description h4 a")
    PRODUCT_PRICE = (By.CSS_SELECTOR, ".description > .price span")
    CAROUSEL_ITEM = (By.CSS_SELECTOR, "#carousel-banner-0")
    HEADER = (By.CSS_SELECTOR, "#top")
    MONEY_LIST = (By.CSS_SELECTOR, "#top > div.row .product-thumb")
    SEARCH_BUTTON = (By.CSS_SELECTOR, "#search button")
    SEARCH_FIELD = (By.XPATH, "//input[@name='search']")
    SEARCH_ICON = (By.XPATH, "//i[@class='fa-solid fa-magnifying-glass']")
    SEARCH_RESULT_HEADER = (By.XPATH, "//*[@id='content']/h1")
    MACBOOK_IMAGE_LINK = (By.XPATH, "//img[@title='MacBook']")
    MACBOOK_INVALID_IMAGE_LINK = (By.XPATH, "//img[@title='MacBook Pro Max']")
    IMAGE_COUNTER = (By.XPATH, "//div[text()='2 of 5']")
    POPUP_CLOSE_BUTTON = (By.XPATH, "/html/body/div[2]/div/button[2]")
    CURRENCY_DROPDOWN = (By.CLASS_NAME, "dropdown-toggle")
    EUR_CURRENCY_LINK = (By.XPATH, "//a[@href='EUR']")
    CURRENCY_SYMBOL = (By.XPATH, "//strong")
    DESKTOPS_LINK = (By.XPATH, f"//a[@href='{BasePage.BASE_URL}/en-gb/catalog/desktops']")
    PC_LINK = (By.XPATH, f"//a[@href='{BasePage.BASE_URL}/en-gb/catalog/desktops/pc']")
    CONTENT_TEXT = (By.ID, "content")

    def __init__(self, driver):
        super().__init__(driver)
        self.driver = driver

    def open_main_page(self):
        self.driver.get(self.BASE_URL)

    def enter_search_text(self, text):
        self.driver.find_element(*self.SEARCH_FIELD).send_keys(text)

    def click_search_button(self):
        self.driver.find_element(*self.SEARCH_ICON).click()

    def get_search_result_header_text(self):
        return self.driver.find_element(*self.SEARCH_RESULT_HEADER).text

    def open_macbook_page(self):
        self.driver.get(f"{self.BASE_URL}/en-gb/product/macbook")

    def click_macbook_image(self):
        self.driver.find_element(*self.MACBOOK_IMAGE_LINK).click()

    def click_invalid_macbook_image(self):
        self.driver.find_element(*self.MACBOOK_INVALID_IMAGE_LINK).click()

    def close_popup(self):
        self.driver.find_element(*self.POPUP_CLOSE_BUTTON).click()

    def get_image_counter_text(self):
        return self.driver.find_element(*self.IMAGE_COUNTER).text

    def switch_to_eur_currency(self):
        self.driver.find_element(*self.CURRENCY_DROPDOWN).click()
        self.driver.find_element(*self.EUR_CURRENCY_LINK).click()

    def get_currency_symbol_text(self):
        return self.driver.find_element(*self.CURRENCY_SYMBOL).text

    def navigate_to_pc_page(self):
        self.driver.find_element(*self.DESKTOPS_LINK).click()
        self.driver.find_element(*self.PC_LINK).click()

    def get_content_text(self):
        return self.driver.find_element(*self.CONTENT_TEXT).text


class TestMainPage:
    @pytest.fixture(scope="session")
    def main_page(self, driver):
        return MainPage(driver)

    @allure.feature("Главная страница")
    @allure.title("Тест поиска")
    @pytest.mark.parametrize("search_text", ["MacBook", "iPhone"])
    def test_search(self, main_page, search_text):
        main_page.open_main_page()
        main_page.enter_search_text(search_text)
        main_page.click_search_button()

        expected = f"Search - {search_text}"
        actual = main_page.get_search_result_header_text()
        assert actual == expected

    @allure.feature("Главная страница")
    @allure.title("Негативный тест поиска")
    @pytest.mark.parametrize("search_text", ["Invalid Product Name"])
    def test_search_negative(self, main_page, search_text):
        main_page.open_main_page()
        main_page.enter_search_text(search_text)
        main_page.click_search_button()

        expected = f"Search - MacBook"
        actual = main_page.get_search_result_header_text()
        assert actual != expected

    @allure.feature("Главная страница")
    @allure.title("Тест переключения фоток")
    def test_switch_screens(self, main_page):
        main_page.open_macbook_page()
        main_page.click_macbook_image()
        main_page.close_popup()

        expected = "2 of 5"
        actual = main_page.get_image_counter_text()
        assert actual == expected

    @allure.feature("Главная страница")
    @allure.title("Негативный тест переключения фоток")
    def test_switch_screens_negative(self, main_page):
        main_page.open_macbook_page()
        try:
            main_page.click_invalid_macbook_image()
        except NoSuchElementException:
            assert True
        else:
            assert False, "NoSuchElementException не выдано"

    @allure.feature("Главная страница")
    @allure.title("Тест переключения валюты")
    def test_switch_currency(self, main_page):
        main_page.open_main_page()
        main_page.switch_to_eur_currency()

        expected = "€"
        actual = main_page.get_currency_symbol_text()
        assert actual == expected

    @allure.feature("Главная страница")
    @allure.title("Негативный тест переключения валюты")
    def test_switch_currency_negative(self, main_page):
        main_page.open_main_page()
        main_page.switch_to_eur_currency()

        expected = "$"
        actual = main_page.get_currency_symbol_text()
        assert actual != expected

    @allure.feature("Главная страница")
    @allure.title("Тест страницы PC")
    def test_page_pc(self, main_page):
        main_page.open_main_page()
        main_page.navigate_to_pc_page()

        expected = ("Desktops\n"
                    "Example of category description text\n"
                    "There are no products to list in this category.\n"
                    "Continue")
        actual = main_page.get_content_text()
        assert actual == expected

    @allure.feature("Главная страница")
    @allure.title("Негативный тест страницы PC")
    def test_page_pc_negative(self, main_page):
        main_page.open_main_page()
        main_page.navigate_to_pc_page()

        expected = "There are nothing."
        actual = main_page.get_content_text()
        assert actual != expected


@pytest.fixture(scope="session")
def driver():
    driver = webdriver.Firefox()
    driver.maximize_window()
    yield driver
    driver.quit()

Overwriting test_Main_Page.py


In [48]:
!pytest --alluredir=kt4-allure-results test_Main_Page.py

Test session starts (platform: darwin, Python 3.9.6, pytest 8.3.4, pytest-sugar 1.0.0)
rootdir: /Users/valdor/Projects/PycharmProjects/WebTesting/KT4
plugins: cov-6.0.0, sugar-1.0.0, anyio-4.8.0, allure-pytest-2.13.5
collected 9 items                                                              

 test_Main_Page.py ✓✓✓✓✓✓✓✓✓                                     100% ██████████

Results (13.86s):
       9 passed


### Тесты для страницы продукта

In [49]:
%%file test_Product_Page.py
from selenium.webdriver.remote.webelement import WebElement
from selenium.common import StaleElementReferenceException, NoSuchFrameException
from selenium.common import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import pytest
from Base_Page import BasePage
import allure


class ProductPage(BasePage):
    PHONES_PAGE_LINK = (By.XPATH, "//*[@id=\"narbar-menu\"]/ul/li[6]/a")
    CAMERAS_PAGE_LINK = (By.XPATH, "//*[@id=\"narbar-menu\"]/ul/li[7]/a")
    TABLETS_PAGE_LINK = (By.XPATH, "//*[@id=\"narbar-menu\"]/ul/li[4]/a")

    WISHLIST_ON_PRODUCT_PAGE_BUTTON = (By.XPATH, "//*[@id=\"content\"]/div[1]/div[2]/form/div/button[1]")
    PRODUCT_NAME = (By.XPATH, "//*[@id=\"content\"]/div[2]/div/div/div[1]/a/img")

    OPTIONAL_SUBMENU = (By.XPATH, "//*[@id=\"input-option-226\"]")
    RED_COLOR = (By.XPATH, "//*[@id=\"input-option-226\"]/option[2]")
    ADD_TO_CART_BUTTON = (By.XPATH, "//*[@id=\"button-cart\"]")

    REVIEWS_BUTTON = (By.XPATH, "//*[@id=\"content\"]/ul/li[3]/a")
    CONTINUE_REVIEW_BUTTON = (By.XPATH, "//*[@id=\"button-review\"]")
    REVIEW_NAME_FIELD = (By.XPATH, "//*[@id=\"input-name\"]")
    REVIEW_TEXT = (By.XPATH, "//*[@id=\"input-text\"]")
    REVIEW_RATING = (By.XPATH, "//*[@id=\"input-rating\"]")

    LOGIN_ALERT = (By.XPATH, "//*[@id=\"alert\"]/div")
    CART_ALERT = (By.XPATH, "//*[@id=\"alert\"]/div")

    def __init__(self, driver):
        super().__init__(driver)

    def open_main_page(self):
        self.driver.get(self.BASE_URL)

    def click_product_name(self, product_index, frame_locator=None, max_attempts=3):
        for attempt in range(max_attempts):
            try:
                if frame_locator:
                    try:
                        frame = WebDriverWait(self.driver, 3).until(
                            EC.presence_of_element_located(frame_locator))
                        self.driver.switch_to.frame(frame)
                    except (TimeoutException, NoSuchFrameException) as e:
                        print(f"Ошибка при переключении на фрейм {frame_locator}: {e}")
                        pass

                products = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_all_elements_located(self.PRODUCT_NAME))
                element = products[product_index]
                self.driver.execute_script("arguments[0].scrollIntoView();", element)
                WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable(element))

                try:
                    element.click()
                    return
                except StaleElementReferenceException as e:
                    print(f"StaleElementReferenceException при клике (попытка {attempt + 1}): {e}")
                    pass

            except (TimeoutException, StaleElementReferenceException) as e:
                print(f"Ошибка (попытка {attempt + 1}): {e}")
                pass
            finally:
                try:
                    self.driver.switch_to.default_content()
                except Exception as e:
                    print(f"Не удалось вернуться к основному контенту: {e}")

        print(f"Не удалось кликнуть на элемент с индексом {product_index} после {max_attempts} попыток")

    def find_and_click_element(self, locator: tuple[str, str], frame_locator: tuple[str, str] = None, max_attempts=3):
        for attempt in range(max_attempts):
            try:
                if frame_locator:
                    try:
                        frame = WebDriverWait(self.driver, 3).until(
                            EC.presence_of_element_located(frame_locator))
                        self.driver.switch_to.frame(frame)
                    except (TimeoutException, NoSuchFrameException) as e:
                        print(f"Ошибка при переключении на фрейм {frame_locator}: {e}")
                        pass

                element: WebElement = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located(locator))

                self.driver.execute_script("arguments[0].scrollIntoView();", element)
                WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable(element))

                try:
                    self.driver.execute_script("arguments[0].click();", element)
                    return
                except StaleElementReferenceException as e:
                    print(f"StaleElementReferenceException при клике (попытка {attempt + 1}): {e}")
                    pass

            except (TimeoutException, StaleElementReferenceException) as e:
                print(f"Ошибка (попытка {attempt + 1}): {e}")
                pass
            finally:
                try:
                    if frame_locator:
                        self.driver.switch_to.default_content()
                except Exception as e:
                    print(f"Не удалось вернуться к основному контенту: {e}")

        print(f"Не удалось кликнуть на элемент после {max_attempts} попыток с локатором {locator}")

    def open_phones_page(self):
        self.driver.find_element(*self.PHONES_PAGE_LINK).click()

    def open_cameras_page(self):
        self.driver.find_element(*self.CAMERAS_PAGE_LINK).click()

    def open_optional_options(self):
        self.driver.find_element(*self.OPTIONAL_SUBMENU).click()

    def open_tablets_page(self):
        self.driver.find_element(*self.TABLETS_PAGE_LINK).click()

    def select_red_color(self):
        self.driver.find_element(*self.RED_COLOR).click()

    def get_login_alert_text(self):
        return self.driver.find_element(*self.LOGIN_ALERT).text

    def is_login_alert_displayed(self):
        try:
            self.driver.find_element(*self.LOGIN_ALERT).is_displayed()
            return True
        except Exception:
            return False

    def is_cart_alert_displayed(self):
        try:
            self.driver.find_element(*self.CART_ALERT).is_displayed()
            return True
        except Exception:
            return False

    def is_this_cart_alert(self, expected):
        actual = self.driver.find_element(*self.CART_ALERT).get_attribute("textContent")

        if actual == expected:
            return True
        else:
            print("Текст уведомления не совпадает")
            return False

    def write_a_review(self, name, review_text):
        self.find_and_click_element(self.REVIEW_NAME_FIELD)
        name_field = self.driver.find_element(*self.REVIEW_NAME_FIELD)
        name_field.clear()
        name_field.send_keys(name)
        self.find_and_click_element(self.REVIEW_TEXT)
        text_field = self.driver.find_element(*self.REVIEW_TEXT)
        text_field.clear()
        text_field.send_keys(review_text)

    def rate_the_product(self, rating):
        buttons = self.driver.find_elements(*self.REVIEW_RATING)

        for button in buttons:
            if button.get_attribute("value") == str(rating):
                button.click()
                return
        print(f"Кнопка с рейтингом {rating} не найдена")

    def get_actual_reviewer_name(self):
        return self.driver.find_element(*self.REVIEW_NAME_FIELD).get_attribute("value")

    def get_actual_review_text(self):
        return self.driver.find_element(*self.REVIEW_TEXT).get_attribute("value")


class TestProductPage:
    @pytest.fixture(scope="session")
    def product_page(self, driver):
        return ProductPage(driver)

    @allure.feature("Страница товара")
    @allure.title("Тест добавления товара в вишлист с главной страницы без авторизации")
    @pytest.mark.parametrize("product_index", [0, 1, 2])
    def test_add_to_wishlist_from_main_page_logged_out(self, product_page, product_index):
        product_page.open_main_page()
        product_page.click_product_name(product_index)
        product_page.find_and_click_element(product_page.WISHLIST_ON_PRODUCT_PAGE_BUTTON)
        assert product_page.is_login_alert_displayed()

    @allure.feature("Страница товара")
    @allure.title("Тест добавления в вишлист со страницы продукта без авторизации")
    @pytest.mark.parametrize("product_index", [0, 1, 2])
    def test_add_to_wishlist_from_product_page_logged_out(self, product_page, product_index):
        product_page.open_main_page()
        product_page.click_product_name(product_index)
        product_page.find_and_click_element(product_page.WISHLIST_ON_PRODUCT_PAGE_BUTTON)
        assert product_page.is_login_alert_displayed()

    @allure.feature("Страница товара")
    @allure.title("Тест добавления камеры в корзину")
    @pytest.mark.parametrize("expected", [" Success: You have added Canon EOS 5D to your shopping cart! "])
    def test_add_camera_to_cart(self, product_page, expected):
        product_page.open_main_page()
        product_page.open_cameras_page()
        product_page.click_product_name(product_index=0)
        product_page.open_optional_options()
        product_page.select_red_color()
        product_page.find_and_click_element(product_page.ADD_TO_CART_BUTTON)

        if product_page.is_cart_alert_displayed():
            if product_page.is_this_cart_alert(expected):
                assert True
            else:
                print("Это уведомление не о добавлении в корзину")
                assert False
        else:
            print("Уведомление не появилось")
            assert False

    @allure.feature("Страница товара")
    @allure.title("Тест добавления планшета в корзину")
    @pytest.mark.parametrize("expected", [" Success: You have added Samsung Galaxy Tab 10.1 to your shopping cart! "])
    def test_add_tablet_to_cart(self, product_page, expected):
        product_page.open_main_page()
        product_page.open_tablets_page()
        product_page.click_product_name(product_index=0)
        product_page.find_and_click_element(product_page.ADD_TO_CART_BUTTON)

        if product_page.is_cart_alert_displayed():
            if product_page.is_this_cart_alert(expected):
                assert True
            else:
                print("Это уведомление не о добавлении в корзину")
                assert False
        else:
            print("Уведомление не появилось")
            assert False

    @allure.feature("Страница товара")
    @allure.title("Тест добавления телефона HTC в корзину")
    @pytest.mark.parametrize("expected", [" Success: You have added HTC Touch HD to your shopping cart! "])
    def test_add_htc_phone_to_cart(self, product_page, expected):
        product_page.open_main_page()
        product_page.open_phones_page()
        product_page.click_product_name(product_index=0)
        product_page.find_and_click_element(product_page.ADD_TO_CART_BUTTON)

        if product_page.is_cart_alert_displayed():
            if product_page.is_this_cart_alert(expected):
                assert True
            else:
                print("Это уведомление не о добавлении в корзину")
                assert False
        else:
            print("Уведомление не появилось")
            assert False

    @allure.feature("Страница товара")
    @allure.title("Тест написания отзыва")
    @pytest.mark.parametrize("product_index", [0])
    @pytest.mark.parametrize("name", ["Reviewer"])
    @pytest.mark.parametrize("review_text", ["I really like it!"])
    @pytest.mark.parametrize("rating", [5])
    def test_write_review(self, driver, product_page, product_index, name, review_text, rating):
        product_page.open_main_page()
        product_page.click_product_name(product_index)
        product_page.find_and_click_element(product_page.REVIEWS_BUTTON)
        product_page.write_a_review(name, review_text)
        product_page.rate_the_product(rating)
        product_page.find_and_click_element(product_page.CONTINUE_REVIEW_BUTTON)
        actual_reviewer_name = product_page.get_actual_reviewer_name()
        actual_review_text = product_page.get_actual_review_text()
        assert actual_reviewer_name == name and actual_review_text == review_text


@pytest.fixture(scope="session")
def driver():
    driver = webdriver.Firefox()
    driver.maximize_window()
    yield driver
    driver.quit()

Overwriting test_Product_Page.py


In [50]:
!pytest --alluredir=kt4-allure-results test_Product_Page.py

Test session starts (platform: darwin, Python 3.9.6, pytest 8.3.4, pytest-sugar 1.0.0)
rootdir: /Users/valdor/Projects/PycharmProjects/WebTesting/KT4
plugins: cov-6.0.0, sugar-1.0.0, anyio-4.8.0, allure-pytest-2.13.5
collected 10 items                                                             

 test_Product_Page.py ✓✓✓✓✓✓✓✓✓✓                                 100% ██████████

Results (14.00s):
      10 passed


### Тесты для страницы регистрации

In [51]:
%%file test_Login_Page.py
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import pytest
from Base_Page import BasePage
import allure


class AuthPage(BasePage):
    FIRSTNAME_FIELD = (By.XPATH, "//input[@name='firstname']")
    LASTNAME_FIELD = (By.XPATH, "//input[@name='lastname']")
    EMAIL_FIELD = (By.XPATH, "//*[@id='input-email']")
    PASSWORD_FIELD = (By.XPATH, "//*[@id='input-password']")
    LOGIN_BUTTON = (By.XPATH, "//*[@id='form-login']/div[3]/button")
    CONTINUE_BUTTON = (By.XPATH, "//button[text()='Continue']")
    AGREE_BUTTON = (By.XPATH, "//input[@name='agree']")
    PASSWORD_ERROR = (By.XPATH, "//div[text()='Password must be between 4 and 20 characters!']")

    MY_ACCOUNT_LINK = (By.XPATH, "//span[text()='My Account']")
    REGISTER_LINK = (By.XPATH, "//a[text()='Register']")
    LOGIN_LINK = (By.XPATH, "//a[text()='Login']")

    def __init__(self, driver):
        super().__init__(driver)
        self.driver = driver

    def enter_firstname(self, firstname):
        self.driver.find_element(*self.FIRSTNAME_FIELD).send_keys(firstname)

    def enter_lastname(self, lastname):
        self.driver.find_element(*self.LASTNAME_FIELD).send_keys(lastname)

    def enter_email(self, email):
        self.driver.find_element(*self.EMAIL_FIELD).send_keys(email)

    def enter_password(self, password):
        self.driver.find_element(*self.PASSWORD_FIELD).send_keys(password)

    def click_login(self):
        self.driver.find_element(*self.LOGIN_BUTTON).click()

    def click_agree(self):
        self.driver.find_element(*self.AGREE_BUTTON).send_keys(Keys.SPACE)

    def click_continue(self):
        self.driver.find_element(*self.CONTINUE_BUTTON).click()

    def get_password_error_message(self):
        return self.driver.find_element(*self.PASSWORD_ERROR).text

    def get_account_header(self):
        return self.driver.find_element(By.XPATH, "//*[@id=\"content\"]/h2[1]")

    def get_signup_header(self):
        return self.driver.find_element(By.XPATH, "//*[@id='content']/h1")

    def open_login_page(self):
        self.driver.find_element(*self.MY_ACCOUNT_LINK).click()
        self.driver.find_element(*self.LOGIN_LINK).click()

    def open_register_page(self):
        self.driver.find_element(*self.MY_ACCOUNT_LINK).click()
        self.driver.find_element(*self.REGISTER_LINK).click()


class TestAuth:
    @pytest.fixture(scope="session")
    def auth_page(self, driver):
        return AuthPage(driver)

    @allure.feature("Страница авторизации")
    @allure.title("Тест регистрации")
    def test_signup(self, auth_page):
        auth_page.open_register_page()

        first_name = "Test"
        last_name = "Test"
        email = "123@example.com"
        password = "123123"

        auth_page.enter_firstname(first_name)
        auth_page.enter_lastname(last_name)
        auth_page.enter_email(email)
        auth_page.enter_password(password)
        auth_page.click_agree()
        auth_page.click_continue()

        actual = auth_page.get_signup_header()
        assert actual.is_displayed()

    @allure.feature("Страница авторизации")
    @allure.title("Тест регистрации с некорректным паролем")
    def test_signup_negative(self, auth_page):
        auth_page.open_register_page()

        first_name = "Test"
        last_name = "Test"
        email = "123@example.com"
        password = ""

        auth_page.enter_firstname(first_name)
        auth_page.enter_lastname(last_name)
        auth_page.enter_email(email)
        auth_page.enter_password(password)
        auth_page.click_agree()
        auth_page.click_continue()

        expected = "Password must be between 4 and 20 characters!"
        actual = auth_page.get_password_error_message()
        assert actual == expected

    @allure.feature("Страница авторизации")
    @allure.title("Тест входа")
    def test_login(self, auth_page):
        auth_page.open_login_page()

        # Вводим данные
        auth_page.enter_email("123@example.com")
        auth_page.enter_password("123123")

        auth_page.click_login()
        actual = auth_page.get_account_header()
        assert actual.is_displayed()


@pytest.fixture(scope="session")
def driver():
    driver = webdriver.Firefox()
    driver.maximize_window()
    yield driver
    driver.quit()

Overwriting test_Login_Page.py


In [52]:
!pytest --alluredirkt4-allure-results test_Login_Page.py

ERROR: usage: pytest [options] [file_or_dir] [file_or_dir] [...]
pytest: error: unrecognized arguments: --alluredirkt4-allure-results
  inifile: None
  rootdir: /Users/valdor/Projects/PycharmProjects/WebTesting/KT4



### Тесты для страницы корзины

In [53]:
%%file test_Cart_Page.py
import pytest
from Base_Page import BasePage
from test_Product_Page import ProductPage
from selenium import webdriver
from selenium.webdriver.common.by import By
import allure


class CartPage(BasePage):
    CART_PAGE_LINK = (By.XPATH, "//*[@id=\"top\"]/div/div[2]/ul/li[4]/a/span")
    CART_CONTENT = (By.XPATH, "//*[@id=\"content\"]")

    def __init__(self, driver):
        super().__init__(driver)

    @pytest.fixture(scope="session")
    def product_page(self, driver):
        return ProductPage(driver)

    def open_main_page(self):
        self.driver.get(BasePage.BASE_URL)

    def open_cart_page(self):
        self.driver.find_element(*self.CART_PAGE_LINK).click()

    def get_cart_content(self):
        return self.driver.find_element(*self.CART_CONTENT).text

    def add_htc_phone_to_cart(self, product_page):
        product_page.open_main_page()
        product_page.open_phones_page()
        product_page.click_product_name(product_index=0)
        product_page.find_and_click_element(product_page.ADD_TO_CART_BUTTON)


class TestCartPage:
    @pytest.fixture(scope="session")
    def cart_page(self, driver):
        return CartPage(driver)

    @pytest.fixture(scope="session")
    def product_page(self, driver):
        return ProductPage(driver)

    @allure.feature("Страница корзины")
    @allure.title("Тест уведомления при пустой корзине")
    def test_empty_cart(self, cart_page):
        cart_page.open_main_page()
        cart_page.open_cart_page()
        cart_page.get_cart_content()
        assert cart_page.get_cart_content() == "Shopping Cart\nYour shopping cart is empty!\nContinue"

    @allure.feature("Страница корзины")
    @allure.title("Тест сообщения при не пустой корзине")
    def test_non_empty_cart(self, cart_page, product_page):
        cart_page.add_htc_phone_to_cart(product_page)
        cart_page.open_main_page()
        cart_page.open_cart_page()
        actual_cart_content = cart_page.get_cart_content()
        assert actual_cart_content == "Shopping Cart (0.15kg)\nImage Product Name Model Quantity Unit Price Total\nHTC Touch HD\n- Reward Points: 400 Product 1\n$122.00 $122.00\nSub-Total $100.00\nEco Tax (-2.00) $2.00\nVAT (20%) $20.00\nTotal $122.00\nWhat would you like to do next?\nChoose if you have a discount code or reward points you want to use or would like to estimate your delivery cost.\nEstimate Shipping & Taxes\nUse Coupon Code\nUse Gift Certificate\n\nContinue Shopping\nCheckout"


@pytest.fixture(scope="session")
def driver():
    driver = webdriver.Firefox()
    driver.maximize_window()
    yield driver
    driver.quit()

Overwriting test_Cart_Page.py


In [54]:
!pytest --alluredir=kt4-allure-results test_Cart_Page.py
!allure generate kt4-allure-results -o kt4-allure-report --clean --single-file

Test session starts (platform: darwin, Python 3.9.6, pytest 8.3.4, pytest-sugar 1.0.0)
rootdir: /Users/valdor/Projects/PycharmProjects/WebTesting/KT4
plugins: cov-6.0.0, sugar-1.0.0, anyio-4.8.0, allure-pytest-2.13.5
collected 2 items                                                              

 test_Cart_Page.py ✓✓                                            100% ██████████

Results (5.68s):
       2 passed
Report successfully generated to kt4-allure-report
